In [1]:
import sys
import os

In [2]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [3]:
import json

In [10]:
with open("../dataset/PIZZA_train_sampled.json", 'r') as infile, open("../dataset/PIZZA_train_2.json", 'w') as outfile:

    line_count = 0
    processed_count = 0
    batch = []

    for line in infile:

        instance = json.loads(line)
        top_field = instance.get(f"train.TOP", "")

        labels = []
        orders = []

        stack = []  # To track all special tokens

        i = 0
        flag = False
        while i < len(top_field):
            char = top_field[i]
            if char == '(':
                # Found the start of a new group
                i += 1
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                entity = top_field[group_start:i]
                stack.append(entity)
                if entity in {"PIZZAORDER", "DRINKORDER"}:
                    flag = True
                    orders.append([])
                    labels.append([])
            elif char == ')':
                # End of the current entity
                if stack and stack[-1] in {"PIZZAORDER", "DRINKORDER"}:
                    flag = False
                if stack:
                    stack.pop()
                i += 1
            elif char == ' ':
                # Skip spaces
                i += 1
            else:
                # Process a token
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                word = top_field[group_start:i]

                if flag:
                    orders[-1].append(word)
                    label = '-'.join(stack[2:])
                    if label == "": label = "OTHER"
                    labels[-1].append(label)
        
        for i in range(len(orders)):
            item = {
                "train.SRC": ' '.join(orders[i]),
                "train.LABELS": labels[i]
            }
            outfile.write(json.dumps(item) + '\n')


In [12]:
with open("../dataset/PIZZA_train_2.json", 'r') as infile:
    unique_labels = set()

    for line in infile:
        instance = json.loads(line)
        labels_field = instance.get(f"train.LABELS", "")
        for label in labels_field: unique_labels.add(label)


print(unique_labels)

    

{'DRINKTYPE', 'OTHER', 'TOPPING', 'VOLUME', 'NOT-TOPPING', 'SIZE', 'CONTAINERTYPE', 'NOT-COMPLEX_TOPPING-QUANTITY', 'NOT', 'STYLE', 'NOT-COMPLEX_TOPPING-TOPPING', 'COMPLEX_TOPPING-QUANTITY', 'NOT-STYLE', 'COMPLEX_TOPPING-TOPPING', 'NUMBER'}


In [13]:
with open("../dataset/PIZZA_test.json", 'r') as infile, open("../dataset/PIZZA_test_2.json", 'w') as outfile:

    line_count = 0
    processed_count = 0
    batch = []

    for line in infile:

        instance = json.loads(line)
        top_field = instance.get(f"test.TOP", "")

        labels = []
        orders = []

        stack = []  # To track all special tokens

        i = 0
        flag = False
        while i < len(top_field):
            char = top_field[i]
            if char == '(':
                # Found the start of a new group
                i += 1
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                entity = top_field[group_start:i]
                stack.append(entity)
                if entity in {"PIZZAORDER", "DRINKORDER"}:
                    flag = True
                    orders.append([])
                    labels.append([])
            elif char == ')':
                # End of the current entity
                if stack and stack[-1] in {"PIZZAORDER", "DRINKORDER"}:
                    flag = False
                if stack:
                    stack.pop()
                i += 1
            elif char == ' ':
                # Skip spaces
                i += 1
            else:
                # Process a token
                group_start = i
                while i < len(top_field) and top_field[i] != ' ':
                    i += 1
                word = top_field[group_start:i]

                if flag:
                    orders[-1].append(word)
                    label = '-'.join(stack[2:])
                    if label == "": label = "OTHER"
                    labels[-1].append(label)
        
        for i in range(len(orders)):
            item = {
                "test.SRC": ' '.join(orders[i]),
                "test.LABELS": labels[i]
            }
            outfile.write(json.dumps(item) + '\n')


In [14]:
with open("../dataset/PIZZA_test_2.json", 'r') as infile:
    unique_labels = set()

    for line in infile:
        instance = json.loads(line)
        labels_field = instance.get(f"test.LABELS", "")
        for label in labels_field: unique_labels.add(label)


print(unique_labels)

    

{'DRINKTYPE', 'OTHER', 'TOPPING', 'VOLUME', 'NOT-TOPPING', 'SIZE', 'CONTAINERTYPE', 'NOT-COMPLEX_TOPPING-QUANTITY', 'STYLE', 'NOT-COMPLEX_TOPPING-TOPPING', 'COMPLEX_TOPPING-QUANTITY', 'NOT-STYLE', 'COMPLEX_TOPPING-TOPPING', 'NUMBER'}


In [ ]:
{'DRINKTYPE', 'OTHER', 'TOPPING', 'VOLUME', 'NOT-TOPPING', 'SIZE', 'CONTAINERTYPE', 'NOT-COMPLEX_TOPPING-QUANTITY', 'NOT', 'STYLE', 'NOT-COMPLEX_TOPPING-TOPPING', 'COMPLEX_TOPPING-QUANTITY', 'NOT-STYLE', 'COMPLEX_TOPPING-TOPPING', 'NUMBER'}